## Data Ingestion

first step of rag pipeline

pdf >> parshing >> document >> chunking >> Embedding >> Vector DB

In [ ]:
## Document Data Structure
#using langchain (core comp : page content(str),metadata(dict (source file name , page , time )) )

from langchain_core.documents import Document

In [2]:
#meta data help in content filtering
doc=Document(
    page_content="hi , this is example for rag pipeline ",
    metadata={
        "source":"example.txt",
        "pages":1,
        "Data":"2026-01-01"
    }
)
doc

Document(metadata={'source': 'example.txt', 'pages': 1, 'Data': '2026-01-01'}, page_content='hi , this is example for rag pipeline ')

In [3]:
##creating simple text file
import os
os.makedirs("../Data/TextFiles",exist_ok=True)

In [4]:
sample_texts={
    "../data/TextFiles/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
"../data/TextFiles/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created!")

✅ Sample text files created!


In [5]:
#reading text file to make document
#textloader
from langchain_community.document_loaders import TextLoader

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
loader=TextLoader("../data/TextFiles/python_intro.txt",encoding="utf-8")
document=loader.load() # load() convert file Data into document
print(document)

[Document(metadata={'source': '../data/TextFiles/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [7]:
## directory loader
#reading all files present at a given place
from langchain_community.document_loaders import DirectoryLoader
dir_loader=DirectoryLoader( #specify config of loader
    "../Data/TextFiles", #file dir
    glob="**/*.txt", #pattern to match
    show_progress=True,
    loader_kwargs={'encoding':"utf-8"},
    loader_cls=TextLoader
)

In [8]:
dirdoc=dir_loader.load()
print(dirdoc)

100%|██████████| 2/2 [00:00<00:00, 2005.88it/s]

[Document(metadata={'source': '..\\Data\\TextFiles\\machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n\n\n    '), Document(metadata={'source': '..\\Data\\TextFiles\\python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular

In [9]:
# for pdf data loading
from langchain_community.document_loaders import PyPDFLoader
pdf_loader=DirectoryLoader( #specify config of loader
    "../Data/pdf", #file dir
    glob="**/*.pdf", #pattern to match
    show_progress=True,
    loader_cls=PyPDFLoader
)

pdfdoc=pdf_loader.load()
pdfdoc


100%|██████████| 2/2 [00:19<00:00,  9.74s/it]


[Document(metadata={'producer': '3-Heights(TM) PDF Optimization Shell 5.9.1.5 (http://www.pdf-tools.com)', 'creator': 'AH CSS Formatter V6.2 MR4 for Linux64 : 6.2.6.18551 (2014/09/24 15:00JST)', 'creationdate': '2016-09-21T13:04:39+00:00', 'author': 'Andreas C. Müller and Sarah Guido', 'title': 'Introduction to Machine Learning with Python', 'trapped': '/False', 'moddate': '2020-08-19T07:09:16+02:00', 'source': '..\\Data\\pdf\\Introduction to Machine Learning with Python ( PDFDrive.com )-min.pdf', 'total_pages': 392, 'page': 0, 'page_label': 'Cover'}, page_content='Andreas C. Müller & Sarah Guido\nIntroduction to \nMachine \nLearning  \nwith P y t h o n   \nA GUIDE FOR DATA SCIENTISTS'),
 Document(metadata={'producer': '3-Heights(TM) PDF Optimization Shell 5.9.1.5 (http://www.pdf-tools.com)', 'creator': 'AH CSS Formatter V6.2 MR4 for Linux64 : 6.2.6.18551 (2014/09/24 15:00JST)', 'creationdate': '2016-09-21T13:04:39+00:00', 'author': 'Andreas C. Müller and Sarah Guido', 'title': 'Intr

In [10]:
#chunking

from pathlib import Path
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")


Found 2 PDF files to process

Processing: Introduction to Machine Learning with Python ( PDFDrive.com )-min.pdf
  ✓ Loaded 392 pages

Processing: Machine Learning ( etc.) (z-lib.org).pdf
  ✓ Loaded 741 pages

Total documents loaded: 1133


In [13]:
# NEW
from langchain_text_splitters import RecursiveCharacterTextSplitter

### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


chunks=split_documents(all_pdf_documents)
chunks

Split 1133 documents into 2094 chunks

Example chunk:
Content: Andreas C. Müller & Sarah Guido
Introduction to 
Machine 
Learning  
with P y t h o n   
A GUIDE FOR DATA SCIENTISTS...
Metadata: {'producer': '3-Heights(TM) PDF Optimization Shell 5.9.1.5 (http://www.pdf-tools.com)', 'creator': 'AH CSS Formatter V6.2 MR4 for Linux64 : 6.2.6.18551 (2014/09/24 15:00JST)', 'creationdate': '2016-09-21T13:04:39+00:00', 'author': 'Andreas C. Müller and Sarah Guido', 'title': 'Introduction to Machine Learning with Python', 'trapped': '/False', 'moddate': '2020-08-19T07:09:16+02:00', 'source': '..\\data\\pdf\\Introduction to Machine Learning with Python ( PDFDrive.com )-min.pdf', 'total_pages': 392, 'page': 0, 'page_label': 'Cover', 'source_file': 'Introduction to Machine Learning with Python ( PDFDrive.com )-min.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': '3-Heights(TM) PDF Optimization Shell 5.9.1.5 (http://www.pdf-tools.com)', 'creator': 'AH CSS Formatter V6.2 MR4 for Linux64 : 6.2.6.18551 (2014/09/24 15:00JST)', 'creationdate': '2016-09-21T13:04:39+00:00', 'author': 'Andreas C. Müller and Sarah Guido', 'title': 'Introduction to Machine Learning with Python', 'trapped': '/False', 'moddate': '2020-08-19T07:09:16+02:00', 'source': '..\\data\\pdf\\Introduction to Machine Learning with Python ( PDFDrive.com )-min.pdf', 'total_pages': 392, 'page': 0, 'page_label': 'Cover', 'source_file': 'Introduction to Machine Learning with Python ( PDFDrive.com )-min.pdf', 'file_type': 'pdf'}, page_content='Andreas C. Müller & Sarah Guido\nIntroduction to \nMachine \nLearning  \nwith P y t h o n   \nA GUIDE FOR DATA SCIENTISTS'),
 Document(metadata={'producer': '3-Heights(TM) PDF Optimization Shell 5.9.1.5 (http://www.pdf-tools.com)', 'creator': 'AH CSS Formatter V6.2 MR4 for Linux64 : 6.2.6.18551 (2014/09/24 15:00JST)',

In [14]:
#embedding and vector DB
import numpy as np
from sentence_transformers import SentenceTransformer
from typing import List,Any,Tuple,Dict
import uuid
from sklearn.metrics.pairwise import cosine_similarity
import chromadb
from chromadb.config import Settings


In [15]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager


Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


## Vector Store


In [16]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 6282


In [17]:
embedings=embedding_manager.generate_embeddings([doc.page_content for doc in chunks])
vectorstore.add_documents(chunks,embedings)

Generating embeddings for 2094 texts...


Batches: 100%|██████████| 66/66 [00:04<00:00, 14.14it/s]


Generated embeddings with shape: (2094, 384)
Adding 2094 documents to vector store...
Successfully added 2094 documents to vector store
Total documents in collection: 8376


In [18]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)



In [19]:
rag_retriever.retrieve("The choice of the model used to solve a specific learning problem is a human task")

Retrieving documents for query: 'The choice of the model used to solve a specific learning problem is a human task'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 70.35it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_dca8b02e_2019',
  'content': 'Model Question Paper-1\n<< NAME OF THE INSTITUTE >>\nDEPARTMENT OF << XXX >>\n \nFinal Semester Examination, August 2018\nSUBJECT NAME: BASICS OF MACHINE\nLEARNING\nSUBJECT CODE: <XXX>\nTIME: 3 HOUR\nFULL MARKS: 80\nPART A (Multiple Choice Type Questions)\n10 X 1 = 10\n1. Answer any ten questions.\n1. A computer program is said to learn from __________ with\nrespect to some class of tasks T and performance measure P, if its\nperformance at tasks in T, as measured by P, improves with it.\n1. Training\n2. Experience\n3. Database\n4. Algorithm\n2. This type of learning to be used when there is no idea about the\nclass or label of a particular data\n1. Supervised learning algorithm',
  'metadata': {'source_file': 'Machine Learning ( etc.) (z-lib.org).pdf',
   'content_length': 664,
   'source': '..\\data\\pdf\\Machine Learning ( etc.) (z-lib.org).pdf',
   'doc_index': 2019,
   'page': 699,
   'file_type': 'pdf'},
  'similarity_score': 0.1644531488

## integration vectordb context pipeline with LLM


In [25]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()
groq_api_key=os.getenv('groq_api_key')

chatLLM = ChatGroq(groq_api_key=groq_api_key,model_name="openai/gpt-oss-120b",temperature=0.1,max_tokens=1024)

In [26]:
def rag_simple(query :str,retriever,llm,top_k=3):
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context :
        return ("no relevant context found")

    prompt = f"""Based on this context: {context}

Question: {query}

Answer:"""
    
    response=llm.invoke(prompt.format(context,query))
    return response.content

In [27]:
answer=rag_simple("what is meant by The choice of the model used to solve a specific learning problem is a human task",rag_retriever,chatLLM)
print(answer)

Retrieving documents for query: 'what is meant by The choice of the model used to solve a specific learning problem is a human task'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 70.77it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


**What the sentence means**

> *“The choice of the model used to solve a specific learning problem is a human task.”*

In machine‑learning projects the **model** is the mathematical or computational structure that will turn raw data into predictions (e.g., a linear regression equation, a decision‑tree, a convolutional neural network, etc.). Selecting which of those possible structures to use is **not done automatically by the data itself**; it is a decision that must be made by a person (or a team of people) who understands:

| Aspect | Why a human must decide |
|--------|--------------------------|
| **Problem definition** | Different problems require different kinds of outputs (binary class, multi‑class, real‑valued score, ranking, etc.). A human translates the business or scientific goal into a learning task (classification, regression, clustering, …). |
| **Data characteristics** | The size, dimensionality, noise level, missing‑value pattern, and type of features (images, text, tab

In [29]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("what is meant by The choice of the model used to solve a specific learning problem is a human task", rag_retriever,chatLLM, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'what is meant by The choice of the model used to solve a specific learning problem is a human task'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 62.87it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


Answer: It means that deciding **which kind of model (e.g., regression, decision‑tree, neural network, etc.) to apply to a given data set and problem is not done automatically by the learning algorithm itself; it is a judgment made by a human practitioner. The human examines the task (prediction vs. classification), the nature of the target variable (numeric, categorical), the characteristics of the data, performance requirements, interpretability needs, computational constraints, etc., and then selects and configures the appropriate model. In other words, model selection is a design decision that requires human expertise, not a built‑in step of the learning process.
Sources: [{'source': 'Machine Learning ( etc.) (z-lib.org).pdf', 'page': 133, 'score': 0.18903273344039917, 'preview': 'to abstract feature-based concept map from the input data and\ngeneralize the same in the form of a classification algorithm to\ndecide whether a face in the gathering is potentially criminal\nor not.\nWh